In [1]:
import numpy as np
import scipy as sc 
import torch
import matplotlib.pyplot as plt
import imageio

In [2]:
#consider a simple function 

def fun(x): #x is tensor
    return 10*(x[0]**2) + (x[1]**2)

def grad_fun(x): #x is tensor
    return torch.tensor([20*x[0],2*x[1]])


In [3]:
def generate_gif(tracker, filename):
    x_points, y_points,discription = tracker
    images = []
    for i in range(len(x_points)):

        plt.scatter(x_points[0:i+1], y_points[0:i+1],s=14, c="red")
        # Generate a meshgrid for plotting
        xvals = np.linspace(-2, 2, 100)
        yvals = np.linspace(-2, 2, 100)
        X, Y = np.meshgrid(xvals, yvals)
        Z = np.zeros_like(X)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i, j] = fun(torch.tensor([X[i, j], Y[i, j]]))

        # Plot the contour lines and tracing
        plt.contour(X, Y, Z, levels=12)

        # Label the axes and title
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        plt.text(0, 1.8,discription, fontsize = 9)
        plt.savefig(f"temp_{i}.png",transparent = False, facecolor = 'white')
        im =imageio.v2.imread(f'temp_{i}.png')
        images.append(im)
        plt.close()
    imageio.mimsave(filename, images, fps=1 ,loop = 10)

In [4]:
def vanilla_SGD(alpha = 0.01,max_epochs=100):
    x_points = []
    y_points = []
    x = 2*torch.rand(2)  #initialization
    x_points.append(x[0])
    y_points.append(x[1])
    for epoch in range(max_epochs):
#         if(torch.norm(grad_fun(x)) < 1e-6):
#             break;
        x = x - alpha*grad_fun(x) 
        x_points.append(x[0])
        y_points.append(x[1])
    discription = f"vanilla_SGD,lr is {alpha}"
    tracker = [x_points,y_points,discription]
    return tracker

In [5]:
def SGD_moment(alpha = 0.01,gamma=0.9,max_epochs=100):
    x_points = []
    y_points = []
    x = 2*torch.rand(2)
    update =torch.zeros(2) #initialization
    x_points.append(x[0])
    y_points.append(x[1])
    for epoch in range(max_epochs):
#         if(torch.norm(grad_fun(x)) < 1e-6):
#             break;
        update = gamma*update + alpha*grad_fun(x)
        x = x - update
        x_points.append(x[0])
        y_points.append(x[1])
    discription = f"SGD_moment,lr is {alpha},gamma is {gamma}"
    tracker = [x_points,y_points,discription]
    return tracker
    

# SGD with moment

<p>SGD with momentum (SGD+Momentum) has several benefits over plain SGD (Stochastic Gradient Descent). Here are some of the advantages of using SGD+Momentum:</p>

* Faster Convergence: SGD+Momentum converges faster by leveraging accumulated gradients to accelerate the optimization process and escape shallow local minima.
* Improved Robustness to Noise: The momentum term in SGD+Momentum smooths out noisy gradients, making the optimization process more robust and reliable.
* Enhanced Exploration of Solution Space: SGD+Momentum explores complex solution spaces more efficiently, enabling the optimizer to escape local minima and reach better optima.
* Dampening Oscillations: The momentum term dampens oscillations, preventing overshooting and oscillating around the minimum, resulting in a more stable optimization process.
* Reduced Learning Rate Sensitivity: SGD+Momentum is less sensitive to the choice of learning rate, allowing for larger learning rates without causing instability, reducing the need for fine-tuning.

In [6]:
def SGD_Nesterov_Momentum(alpha = 0.01,gamma=0.9,max_epochs=100):
    x_points = []
    y_points = []
    x = 2*torch.rand(2)
    update =torch.zeros(2) #initialization
    x_points.append(x[0])
    y_points.append(x[1])
    for epoch in range(max_epochs):
#         if(torch.norm(grad_fun(x)) < 1e-6):
#             break;
        x_ahead = x - gamma*update
        update = gamma*update + alpha*grad_fun(x_ahead)
        x = x - update
        x_points.append(x[0])
        y_points.append(x[1])
    
    discription = f"SGD_Nesterov_Momentum,lr is {alpha},gamma is {gamma}"
    tracker = [x_points,y_points,discription]
    return tracker

# SGD_Nesterov_Momentum
<p>benefits of using SGD with Nesterov momentum (Nesterov accelerated gradient or NAG) over 
SGD with momentum:</p>

* Enhanced Gradient Estimation: Nesterov momentum provides a more accurate estimation of the gradient by considering the momentum term and calculating the gradient ahead of the current position.
* Improved Convergence Speed: Nesterov momentum converges faster due to its lookahead gradient estimation, allowing for larger and more effective steps towards the minimum.
* Better Handling of Sharp Turns: Nesterov momentum performs better in areas with sharp turns or high-curvature landscapes by anticipating and adjusting its trajectory ahead of time, preventing overshooting and oscillations.
* Enhanced Robustness to Noise: Nesterov momentum is more robust to noisy gradients, similar to SGD with momentum, by smoothing out irregularities in the gradient updates.
* Theoretical Convergence Guarantees: Nesterov momentum has theoretical guarantees for convex functions, ensuring a faster convergence rate compared to plain SGD and SGD with momentum.

In [7]:
def AdaGrad(alpha = 0.1,epsilon = 0.01,max_epochs=100):
    x_points = []
    y_points = []
    x = 2*torch.rand(2)
    v =torch.zeros(1) #initialization
    x_points.append(x[0])
    y_points.append(x[1])
    for epoch in range(max_epochs):
#         if(torch.norm(grad_fun(x)) < 1e-6):
#             break;
        v = v + (grad_fun(x)@grad_fun(x)) 
        x = x - (alpha/(torch.sqrt(v+epsilon)))*grad_fun(x)
        x_points.append(x[0])
        y_points.append(x[1])
    discription = f"AdaGrad,lr is {alpha},epsilon is {epsilon}"
    tracker = [x_points,y_points,discription]
    return tracker

# AdaGrad

<p>benefits of using AdaGrad (Adaptive Gradient) optimization algorithm:</p>

* Adaptive Learning Rates: AdaGrad adapts the learning rate for each parameter based on their historical gradients. It assigns larger learning rates to parameters with infrequent updates and smaller learning rates to parameters with frequent updates. This adaptive learning rate scheme allows AdaGrad to effectively navigate different parts of the optimization landscape, leading to efficient convergence.
* Automatic Scaling: AdaGrad automatically scales the learning rate based on the magnitude of gradients. It divides the learning rate by the square root of the sum of squared gradients, which effectively normalizes the updates for each parameter. This feature is beneficial when dealing with data with varying scales or when optimizing models with parameters of significantly different magnitudes.
* Robustness to Sparse Features: AdaGrad performs well in scenarios with sparse features. Since AdaGrad accumulates squared gradients over time, it naturally emphasizes features with sparse updates by allowing larger learning rates for those features. This property makes AdaGrad particularly suitable for problems such as natural language processing and recommender systems, where data often exhibits sparsity.
* Convergence on Steep Directions: AdaGrad tends to converge faster on steep directions in the optimization landscape. By accumulating gradients over time, AdaGrad emphasizes updates along dimensions with large gradients, which can be helpful in optimizing models with steep or elongated valleys.
* Reduced Need for Learning Rate Tuning: AdaGrad's adaptive learning rate scheme reduces the need for manual tuning of the learning rate. It automatically adjusts the learning rate based on the gradient history, mitigating the risk of using an overly large or small learning rate. This feature saves time and effort in the hyperparameter tuning process.

In [8]:
def RMSProp(alpha = 0.1,beta=0.5,epsilon = 0.01,max_epochs=100):
    x_points = []
    y_points = []
    x = 2*torch.rand(2)
    v =torch.zeros(1) #initialization
    x_points.append(x[0])
    y_points.append(x[1])
    for epoch in range(max_epochs):
#         if(torch.norm(grad_fun(x)) < 1e-6):
#             break;
        v = beta*v + (1-beta)*(grad_fun(x)@grad_fun(x))
        x = x - (alpha/(torch.sqrt(v+epsilon)))*grad_fun(x)
        x_points.append(x[0])
        y_points.append(x[1])
    discription = f"AdaGrad,lr is {alpha},beta is {beta},epsilon is {epsilon}"
    tracker = [x_points,y_points,discription]
    return tracker

# RMSProp

<p>RMSProp (Root Mean Square Propagation) shares some similarities with AdaGrad but incorporates a modification that addresses some of AdaGrad's limitations. Here are some benefits of using RMSProp over AdaGrad:</p>

* Adaptive learning rates with momentum: Like AdaGrad, RMSProp adapts the learning rate for each parameter based on the historical gradients. However, RMSProp also introduces a momentum term that helps accelerate the convergence process. By incorporating momentum, RMSProp is able to retain information about the previous gradients and make more informed updates. This momentum term helps smooth out the optimization process, leading to faster convergence.

* Better handling of velocities: AdaGrad treats all historical gradients equally, which may not be ideal in non-stationary environments where the optimal solution changes over time. RMSProp mitigates this issue by using an exponentially weighted moving average, which gives more weight to recent gradients and less weight to past gradients. This adaptability to changing environments allows RMSProp to quickly adjust the learning rate based on current gradients, facilitating more effective optimization in dynamic scenarios.



In [9]:
r2 = vanilla_SGD(alpha = 0.01,max_epochs=100)                     # generate_gif(r2,"r2.gif")
r3 = SGD_moment(alpha = 0.01,gamma=0.9,max_epochs=100)            # generate_gif(r3,"r3.gif")
r4 = SGD_Nesterov_Momentum(alpha = 0.01,gamma=0.9,max_epochs=100) # generate_gif(r4,"r4.gif")
r5 = AdaGrad(alpha = 1,epsilon = 0.05,max_epochs=100)             # generate_gif(r5,"r5.gif")
r6 = RMSProp(alpha = 0.1,beta=0.5,epsilon = 0.01,max_epochs=100)  # generate_gif(r6,"r6.gif")

# <img src="FileName.gif" width="750" align="center"> to display gif in python markdown 

In [10]:
#comparision of techiniques
images = []
for i in range(len(r2[0])):
        plt.plot(r2[0][0:i+1],r2[1][0:i+1])
        plt.plot(r3[0][0:i+1],r3[1][0:i+1])
        plt.plot(r4[0][0:i+1],r4[1][0:i+1])
        plt.plot(r5[0][0:i+1],r5[1][0:i+1])
        plt.plot(r6[0][0:i+1],r6[1][0:i+1])
        plt.legend(["SGD","SGD_moment","SGD_nesterov","Adagrad","RMSprop"])
        # Generate a meshgrid for plotting
        xvals = np.linspace(-2, 2, 100)
        yvals = np.linspace(-2, 2, 100)
        X, Y = np.meshgrid(xvals, yvals)
        Z = np.zeros_like(X)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i, j] = fun(torch.tensor([X[i, j], Y[i, j]]))

        # Plot the contour lines and tracing
        plt.contour(X, Y, Z, levels=12)

        # Label the axes and title
        plt.xlabel("x[0]")
        plt.ylabel("x[1]")
        plt.savefig(f"temp_{i}.png",transparent = False, facecolor = 'white')
        im =imageio.v2.imread(f'temp_{i}.png')
        images.append(im)
        plt.close()
imageio.mimsave("compare.gif",images,fps=1,loop = 10)

# <img src="./FileName.gif" width="750" align="center"> to display gif in python


<img src="./compare.gif" width="750" align="center">

<p>The above gif clearly illustrated the working of all methods described above</p>